<a href="https://colab.research.google.com/github/SivaKarthikPara/CustomerSegRFM/blob/master/Customer_segmentation_based_on_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import os

In [4]:
df = pd.read_csv('CustomerRFM.csv')
# df

In [5]:
#find the coutry in which each of the customer has made purchase
customer_country=df[['Country','CustomerID']].drop_duplicates()
customer_country
#remove null values
df = df[pd.notnull(df['CustomerID'])]
#removes negative values
df.Quantity.min()
df = df[(df['Quantity']>0)]
df.Quantity.min()
#computes totalprice for monetary calculation
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
NOW = dt.datetime(2011,12,10)
#changing date format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [33]:
# from google.colab import drive
# drive.mount('/content/drive')
# dc = df.groupby('CustomerID')
# dc.first()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,TotalPrice
CustomerID,,,,,,,,
12431,536389,22941,CHRISTMAS LIGHTS 10 REINDEER,6,2010-01-12 10:03:00,8.50,Australia,51.00
12583,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-01-12 08:45:00,3.75,France,90.00
13047,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-01-12 08:34:00,1.69,United Kingdom,54.08
13748,536371,22086,PAPER CHAIN KIT 50'S CHRISTMAS,80,2010-01-12 09:00:00,2.55,United Kingdom,204.00
14688,536378,22386,JUMBO BAG PINK POLKADOT,10,2010-01-12 09:37:00,1.95,United Kingdom,19.50
15100,536374,21258,VICTORIAN SEWING BOX LARGE,32,2010-01-12 09:09:00,10.95,United Kingdom,350.40
15291,536376,22114,HOT WATER BOTTLE TEA AND SYMPATHY,48,2010-01-12 09:32:00,3.45,United Kingdom,165.60
15311,536381,22139,RETROSPOT TEA SET CERAMIC 11 PC,23,2010-01-12 09:41:00,4.25,United Kingdom,97.75
16029,536386,84880,WHITE WIRE EGG HOLDER,36,2010-01-12 09:57:00,4.95,United Kingdom,178.20


In [6]:
##calculate the recency, frequency and monetary_value

rfmTable=df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})

##Renaming columns

rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)
rfmTable


In [ ]:

rfmTable['recency'] = rfmTable['recency'].astype(int)
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()
segmented_rfm = rfmTable
segmented_rfm 


In [8]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [9]:
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [21]:
#R score calculation

segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
# F Score calculation
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
#M score calculation
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))


R= np.asarray(segmented_rfm[['r_quartile']])
R_mean=np.mean(R);
F= np.asarray(segmented_rfm[['f_quartile']])
F_mean=np.mean(F);
M= np.asarray(segmented_rfm[['m_quartile']])
M_mean=np.mean(M);
final=R_mean+F_mean+M_mean
with open('output1.txt', 'w') as file:
    file.write(str(final))

In [35]:
#combine RFM score
 
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str)+ segmented_rfm.f_quartile.map(str)+ segmented_rfm.m_quartile.map(str)
 
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12431,696,3,105.60,1,3,4,134
12583,696,20,855.86,1,1,1,111
13047,696,17,366.63,1,2,3,123
13748,696,1,204.00,1,4,4,144
14688,696,19,444.98,1,1,2,112


In [36]:
segmented_rfm[segmented_rfm['RFMScore']=='123'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
13047,696,17,366.63,1,2,3,123
16250,696,14,226.14,1,2,3,123
